In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
data = pd.read_csv('Data_Private/prepared/cleaned_VacMe_Gemeinden_ueber_500.csv')

In [ ]:
data.head()

In [ ]:
data['Erstimpfung ODI Typ'].unique()

In [ ]:
gemeinden = data['Wohngemeinde'].unique()
gemeinden = [g for g in gemeinden if type(g) is str]

In [ ]:
geimpfte_thuner = data[data['Wohngemeinde']=='Thun'].shape[0]

In [ ]:
geimpfte = data[data['Vollstaendiger_Impfschutz'] == 'WAHR']

In [ ]:
geimpfte.shape[0]

In [ ]:
geimpfte[geimpfte['Zweitimpfung Gemeinde'] != geimpfte['Erstimpfung Gemeinde']].shape[0]

In [ ]:
anzahl_heimgeimpfte = geimpfte[geimpfte['Wohngemeinde'] == geimpfte['Erstimpfung Gemeinde']].shape[0]

In [ ]:
anzahl_geimpfte = geimpfte.shape[0]

In [ ]:
print('total score: ', anzahl_heimgeimpfte / anzahl_geimpfte)

In [ ]:
_eigenimpfscore_map = {}
for g in sorted(gemeinden):
    einw = geimpfte[geimpfte['Wohngemeinde']==g]
    heim_impfungen = einw[g == einw['Erstimpfung Gemeinde']].shape[0]
    anz_einw = einw.shape[0]
    score = heim_impfungen / anz_einw
    _eigenimpfscore_map[g] = score

In [ ]:
_hatImpzentrum_map = {}
for g in sorted(gemeinden):
    impfz = geimpfte[(geimpfte['Erstimpfung ODI Typ'] == 'Impfzentrum') & (geimpfte['Erstimpfung Gemeinde'] == g)]
    if impfz.shape[0] > 0:
        _hatImpzentrum_map[g] = True
    else:
        _hatImpzentrum_map[g] = False
print(_hatImpzentrum_map)

In [ ]:
df = pd.DataFrame({'score': _eigenimpfscore_map, 'hatImpfzentrum': _hatImpzentrum_map})

In [ ]:
df.sort_values('score')

In [ ]:
# df.to_csv('Data_Public/eigenimpfscore.csv')

In [ ]:
def clean_gemeindecode():
    portrait = pd.read_csv('Data_Public/generated/regionalportraets2021Gemeinden.csv', sep=";",
                 encoding='Latin1', header=5,
                 parse_dates=True )
    codes = portrait[['Gemeindecode', 'Gemeindename']].dropna()
    return codes.set_index('Gemeindecode')

In [ ]:
codes = clean_gemeindecode()

In [ ]:
codes.head()

In [ ]:
merged = pd.merge(codes, df, left_on='Gemeindename', right_index=True).reset_index()
merged.head()

In [ ]:
merged = merged.rename(columns={'Gemeindecode': 'bfs_nummer', 'score': 'value'})

In [ ]:
eigenscore = merged[['bfs_nummer', 'value']]

In [ ]:
eigenscore

In [ ]:
for k, v in eigenscore.iterrows():
    break
    
    
' '.join([f'INSERT INTO gemeinde_eigenimpf(bfs_nummer, score) VALUES ({v["bfs_nummer"]}, {v["value"]});' for _,v in eigenscore.iterrows()])

In [ ]:
eigenscore.to_csv('')